In [2]:
import pandas as pd
import numpy as np
import math
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from time import sleep as fallasleep
import os, sys

In [9]:
from source.colortools import color_box

In [6]:
import krakenex
from pykrakenapi import KrakenAPI

In [10]:
api = krakenex.API()
k = KrakenAPI(api)
coins = ["ETH", "BTC", "XRP", "ADA", "DOT"]
gapmin= 1440
coins = [coin+"USD" for coin in coins]
df = {}
colormapper = {}
for _,coin in enumerate(coins):
    data, last = k.get_ohlc_data(coin, interval = gapmin, ascending = True)
    df[coin]=data.drop(['time', 'count'], axis=1)
    colormapper[coin] = color_box[_]
    fallasleep(1)


In [28]:
df_close = pd.DataFrame(columns=coins)

for coin in coins:
    df_close[coin] = df[coin]['close']


df_close.isna().sum()

ETHUSD      0
BTCUSD      0
XRPUSD      0
ADAUSD      0
DOTUSD    261
dtype: int64

This is because polkadot was released quite later than other coins

In [31]:
df_close = df_close.dropna()
print(df_close.isna().sum())
display(df_close.head(3))

ETHUSD    0
BTCUSD    0
XRPUSD    0
ADAUSD    0
DOTUSD    0
dtype: int64


,ETHUSD,BTCUSD,XRPUSD,ADAUSD,DOTUSD
dtime,,,,,
2020-08-18,422.62,11957.0,0.30292,0.137218,3.1099
2020-08-19,407.57,11757.4,0.28992,0.129678,2.9175
2020-08-20,416.20,11864.6,0.29246,0.134305,2.9090


In [34]:
df_close_diff = df_close.apply(np.log).diff().dropna()
df_close_diff.head(3)

,ETHUSD,BTCUSD,XRPUSD,ADAUSD,DOTUSD
dtime,,,,,
2020-08-19,-0.036261,-0.016834,-0.043864,-0.056516,-0.063863
2020-08-20,0.020953,0.009076,0.008723,0.035059,-0.002918
2020-08-21,-0.070599,-0.029240,-0.046364,-0.087043,0.056341


### k-medoids clustering for index-tracking
$$
\begin{aligned}
& \underset{x}{\text{minimize}} & & -\frac{\alpha}{2} \cdot x^TDx + \beta \cdot (1^TD) x\\
        &\text{subject to}&& \mathbf{1}^Tx = k\\
        &&& x \in \{0,1\}\\
        &\text{where}&& \alpha = \frac{1}{k-1}, \beta  = \frac{1}{n-1}
\end{aligned}
$$

In [35]:
from gurobipy import Model, GRB

In [157]:
def obj_val(x):
    return -alpha * 0.5 * (x @ D @ x) + beta * (np.ones(n).dot(D) @ x)

In [158]:
m = Model("k-medoids")
x = {}
n = len(coins)
k = 2
alpha = 1 / (k-1)
beta = 1 / (n-1)
D = df_close_diff.corr()
D = 0.5*(1- D)
D = D.apply(np.sqrt)
D = 1 - (-0.5 * D).apply(np.exp)
D = D.values

x = m.addMVar(shape=n,vtype=GRB.BINARY, name=coins)
obj = -alpha * 0.5 * (x @ D @ x) +  beta * (np.ones(n).dot(D) @ x)
m.addConstr(x.sum()==k)
m.setObjective(obj, GRB.MINIMIZE)
m.optimize()

selected = []
gurobi   = []
for v in m.getVars():
    gurobi.append(int(v.x))
    if v.x == 1:
        selected.append(v.VarName)
print(f"Gurobi selected:         {selected}")
print(f"Gurobi optimal solution: {gurobi}")
print(f"Gurobi optimal value:    {obj_val(gurobi)}")

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1 rows, 5 columns and 5 nonzeros
Model fingerprint: 0x19fff7f2
Model has 10 quadratic objective terms
Variable types: 0 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e-01]
  QObjective range [3e-01, 5e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Found heuristic solution: objective 0.2187415
Presolve time: 0.00s
Presolved: 11 rows, 15 columns, 35 nonzeros
Variable types: 0 continuous, 15 integer (15 binary)

Root relaxation: objective -3.989043e-01, 4 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   -0.39890    0    5    0.21874   -0.39890   282%     -    0s
H    0     0                       0.

### D-wave Quantum Annealing Simulator

#### convert our constrained problem to QUBO problem
$$
\begin{aligned}
& \underset{x}{\text{minimize}} & & -\frac{\alpha}{2} (x^T D  x) + \beta (\mathbf{1}^T D)  x +  P \cdot (\mathbf{1}^Tx - k)^2\\
        &&& x \in \{0,1\}\\
\end{aligned}
$$


We approximate $P$ using naive approximation.

The penalty term $P$ multiplies by squared deviations from feasibility, and the unit penalty is observed when we have ($k\pm1$) non-zero entries.

Let $\bar{\delta}$ be an average distance matrix $\bar{D}$;

$$
\bar{\delta} = \begin{cases} 
\frac{1}{n(n-1)}\sum_{i=1}^n\sum_{j=1}^n D_{ij} &\mbox{if } i \neq j \\
0 & \mbox{otherwise } 
\end{cases}
$$

Suppose $x^*$ is a solution containing $k$ nodes, and $x^-$ is a solution containing $k-1$ nodes and $x^+$ is a solution containing $k+1$ nodes. Also let f(x) be the QUBO objective function. We then have the following approximation:

- $f(x^*) \approx \left(\beta\sum_{i=1}^n x^*_i \sum_{j=1}^n \bar{\delta}\right) - \left(\frac{\alpha}{2}\sum_{i=1}^n \sum_{j=1}^n x^*_i x^*_j \bar{\delta}\right) \approx \beta k n \bar{\delta} - \frac{\alpha}{2}k(k-1)\bar{\delta}$

- $f(x^-) \approx \left(\beta\sum_{i=1}^n x^-_i \sum_{j=1}^n \bar{\delta}\right) - \left(\frac{\alpha}{2}\sum_{i=1}^n \sum_{j=1}^n x^-_i x^-_j \bar{\delta}\right) \approx \beta(k-1)n\bar{\delta} - \frac{\alpha}{2}(k-1)(k-2)\bar{\delta}$

- $f(x^+) \approx \left(\beta\sum_{i=1}^n x^+_i \sum_{j=1}^n \bar{\delta}\right) - \left(\frac{\alpha}{2}\sum_{i=1}^n \sum_{j=1}^n x^+_i x^+_j \bar{\delta}\right) \approx \beta(k+1)n\bar{\delta} - \frac{\alpha}{2}(k+1)k\bar{\delta}$

We take $P = max ( f(x^*) - f(x^-) , f(x^*) -f(x^+) ) = max(\frac{\bar{\delta}}{n-1}, (\frac{1}{k-1} - \frac{1}{n-1})\bar{\delta})$

In [103]:
# average distance between nodes
d = np.sum(D)
d /= (n*(n-1))

# penalty P is defined using the equation above
P = max( d/(n-1), (1/(k-1) - 1/(n-1))*d )
P = 2.5

print(f"average distance: {round(d,4)} and penalty term: {round(P,4)}")

average distance: 0.1995 and penalty term: 2.5


Now that we have found the ideal candidate for the penalty term $P$, we formulate our problem as a QUBO problem


$$
\begin{aligned}
&&objective=& -\frac{\alpha}{2} (x^T D  x) + \beta (\mathbf{1}^T D)  x +  P \cdot (\mathbf{1}^Tx - k)^2\\
        &&=& -\frac{\alpha}{2} (x^T D  x) + \beta (\mathbf{1}^T D)  x +  P(\mathbf{1}^Tx - k)^T(\mathbf{1}^Tx - k)\\
        &&=& -\frac{\alpha}{2} (x^T D  x) + \beta (\mathbf{1}^T D)  x +  P(x^T\mathbf{1}\mathbf{1}^Tx - 2k\mathbf{1}^Tx + k^2)\\
        &&=& x^T(P\mathbf{1}\mathbf{1}^T-\frac{\alpha}{2} D)x + (\beta \mathbf{1}^T D - 2k\mathbf{1}^T)  x  + Pk^2
\end{aligned}
$$

We set $Q = (P\mathbf{1}\mathbf{1}^T-\frac{\alpha}{2} D)$ and $b = (\beta \mathbf{1}^T D - 2k\mathbf{1}^T)$. Constant does not effect our optimization, but we will denote our constant as $c=Pk^2$.


In [144]:
Q = P*np.ones((n,n)) - alpha/2*D
b = beta*np.ones(n).dot(D) - 2*P*k*np.ones(n)
c = P*k**2

In [150]:
print(f"number of assets: {n}, quadratic term dim: {Q.shape},\nbias term dim: {b.shape}, constant term: {round(c,4)}")

number of assets: 5, quadratic term dim: (5, 5),
bias term dim: (5,), constant term: 10.0


In [153]:
def qubo_obj(x):
    obj = x.dot(Q).dot(x) + b.dot(x) + c
    return obj

In [169]:
possible_solution = {}
possible_num_solution = 2**n
for sol in range(possible_num_solution):
    binary_sol = format(sol,'b')
    
    if len(binary_sol) < n:
        binary_sol = '0'*(n - len(binary_sol)) + binary_sol
    binary_sol_array = [int(x) for x in binary_sol]
    binary_sol_array = np.array(binary_sol_array)
    possible_solution[binary_sol] = qubo_obj(binary_sol_array)
    non_qubo = obj_val(binary_sol_array)
    non_qubo += (sum(binary_sol_array)-k)**2 * P
    qubo = qubo_obj(binary_sol_array)
    if round(qubo, 8) != round(non_qubo, 8):
        print("hold on, something is wrong here")
        print(f"qubo: {round(qubo, 6)} ||| non-qubo: {round(non_qubo, 6)}")

possible_solution = {k: v for k, v in sorted(possible_solution.items(), key=lambda item: item[1])}
for sol in possible_solution:
    print(f"{sol}: {round(possible_solution[sol],4)}")

10100: 0.1906
01010: 0.1921
00101: 0.1922
01100: 0.1948
01001: 0.1972
10010: 0.2005
00110: 0.2016
00011: 0.2017
10001: 0.2051
11000: 0.2187
01101: 2.4707
01110: 2.4747
00111: 2.475
10101: 2.4863
10110: 2.4909
01011: 2.4942
11100: 2.5093
10011: 2.5225
11010: 2.5333
11001: 2.5432
10000: 2.6823
01000: 2.6944
00001: 2.7011
00010: 2.7014
00100: 2.7181
01111: 9.5497
10111: 9.5859
11110: 9.606
11101: 9.6068
11011: 9.657
00000: 10.0
11111: 21.5027


In [151]:
import dimod

In [175]:
qubo = {}

for i in range(n):
    for j in range(i,n):
        if i == j:
            qubo[(i,j)] = b[i] + Q[i][i]
        else:
            qubo[(i,j)] = 2*Q[i][j]

In [176]:
dict_bqm = dimod.BQM.from_qubo(qubo)
sampler_exact = dimod.ExactSolver()
sampleset = sampler_exact.sample(dict_bqm)
print(sampleset)

    0  1  2  3  4    energy num_oc.
6   1  0  1  0  0 -9.809413       1
12  0  1  0  1  0  -9.80793       1
24  0  0  1  0  1 -9.807824       1
4   0  1  1  0  0 -9.805181       1
28  0  1  0  0  1 -9.802771       1
14  1  0  0  1  0 -9.799467       1
8   0  0  1  1  0   -9.7984       1
16  0  0  0  1  1 -9.798296       1
30  1  0  0  0  1 -9.794935       1
2   1  1  0  0  0 -9.781258       1
27  0  1  1  0  1 -7.529348       1
11  0  1  1  1  0 -7.525349       1
23  0  0  1  1  1 -7.525044       1
25  1  0  1  0  1 -7.513677       1
9   1  0  1  1  0 -7.509051       1
19  0  1  0  1  1 -7.505843       1
5   1  1  1  0  0 -7.490671       1
17  1  0  0  1  1 -7.477477       1
13  1  1  0  1  0 -7.466748       1
29  1  1  0  0  1 -7.456791       1
1   1  0  0  0  0 -7.317665       1
3   0  1  0  0  0 -7.305598       1
31  0  0  0  0  1 -7.298911       1
15  0  0  0  1  0 -7.298645       1
7   0  0  1  0  0  -7.28192       1
20  0  1  1  1  1 -0.450255       1
22  1  0  1  1  1 -0.414054 